In [4]:
from google.colab import drive
drive.mount ('/content/drive/')

Mounted at /content/drive/


In [5]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pylab  as plt
from mpl_toolkits.mplot3d import Axes3D

def iterate_in_chunks(l, n):
    '''Yield successive 'n'-sized chunks from iterable 'l'.
    Note: last chunk will be smaller than l if n doesn't divide l perfectly.
    '''
    for i in range(0, len(l), n):
        yield l[i:i + n]


def unit_cube_grid_point_cloud(resolution, clip_sphere=False):
    '''Returns the center coordinates of each cell of a 3D grid with resolution^3 cells,
    that is placed in the unit-cube.
    If clip_sphere it True it drops the "corner" cells that lie outside the unit-sphere.
    '''
    grid = np.ndarray((resolution, resolution, resolution, 3), np.float32)
    spacing = 1.0 / float(resolution - 1)
    for i in range(resolution):
        for j in range(resolution):
            for k in range(resolution):
                grid[i, j, k, 0] = i * spacing - 0.5
                grid[i, j, k, 1] = j * spacing - 0.5
                grid[i, j, k, 2] = k * spacing - 0.5

    if clip_sphere:
        grid = grid.reshape(-1, 3)
        grid = grid[norm(grid, axis=1) <= 0.5]

    return grid, spacing

def plot_3d_point_cloud(x, y, z, show=True, show_axis=True, in_u_sphere=False, marker='.', s=8, alpha=.8, figsize=(5, 5), elev=10, azim=240, axis=None, title=None, *args, **kwargs):

    if axis is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection='3d')        
    else:
        ax = axis
        fig = axis

    if title is not None:
        plt.title(title)

    sc = ax.scatter(x, y, z, marker=marker, s=s, alpha=alpha, *args, **kwargs)
    ax.view_init(elev=elev, azim=azim)

    if in_u_sphere:
        ax.set_xlim3d(-0.5, 0.5)
        ax.set_ylim3d(-0.5, 0.5)
        ax.set_zlim3d(-0.5, 0.5)
    else:
        miv = 0.7 * np.min([np.min(x), np.min(y), np.min(z)])  # Multiply with 0.7 to squeeze free-space.
        mav = 0.7 * np.max([np.max(x), np.max(y), np.max(z)])
        ax.set_xlim(miv, mav)
        ax.set_ylim(miv, mav)
        ax.set_zlim(miv, mav)
        plt.tight_layout()

    if not show_axis:
        plt.axis('off')

    if 'c' in kwargs:
        plt.colorbar(sc)

    if show:
        plt.show()

    return fig

In [6]:
import tensorflow as tf
import numpy as np
import warnings

from scipy.stats import entropy

try:
    from sklearn.neighbors import NearestNeighbors
except:
    print ('Sklearn module not installed (JSD metric will not work).')

try:    
    from .. external.structural_losses.tf_nndistance import nn_distance
    from .. external.structural_losses.tf_approxmatch import approx_match, match_cost
except:
    print('External Losses (Chamfer-EMD) cannot be loaded. Please install them first.')

def jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28):
    '''Computes the JSD between two sets of point-clouds, as introduced in the paper ```Learning Representations And Generative Models For 3D Point Clouds```.    
    Args:
        sample_pcs: (np.ndarray S1xR2x3) S1 point-clouds, each of R1 points.
        ref_pcs: (np.ndarray S2xR2x3) S2 point-clouds, each of R2 points.
        resolution: (int) grid-resolution. Affects granularity of measurements.
    '''   
    in_unit_sphere = True
    sample_grid_var = entropy_of_occupancy_grid(sample_pcs, resolution, in_unit_sphere)[1]
    ref_grid_var = entropy_of_occupancy_grid(ref_pcs, resolution, in_unit_sphere)[1]
    return jensen_shannon_divergence(sample_grid_var, ref_grid_var)
    
def entropy_of_occupancy_grid(pclouds, grid_resolution, in_sphere=False):
    '''Given a collection of point-clouds, estimate the entropy of the random variables
    corresponding to occupancy-grid activation patterns.
    Inputs:
        pclouds: (numpy array) #point-clouds x points per point-cloud x 3
        grid_resolution (int) size of occupancy grid that will be used.
    '''
    epsilon = 10e-4
    bound = 0.5 + epsilon
    if abs(np.max(pclouds)) > bound or abs(np.min(pclouds)) > bound:
        warnings.warn('Point-clouds are not in unit cube.')

    if in_sphere and np.max(np.sqrt(np.sum(pclouds ** 2, axis=2))) > bound:
        warnings.warn('Point-clouds are not in unit sphere.')

    grid_coordinates, _ = unit_cube_grid_point_cloud(grid_resolution, in_sphere)
    grid_coordinates = grid_coordinates.reshape(-1, 3)
    grid_counters = np.zeros(len(grid_coordinates))
    grid_bernoulli_rvars = np.zeros(len(grid_coordinates))
    nn = NearestNeighbors(n_neighbors=1).fit(grid_coordinates)

    for pc in pclouds:
        _, indices = nn.kneighbors(pc)
        indices = np.squeeze(indices)
        for i in indices:
            grid_counters[i] += 1
        indices = np.unique(indices)
        for i in indices:
            grid_bernoulli_rvars[i] += 1

    acc_entropy = 0.0
    n = float(len(pclouds))
    for g in grid_bernoulli_rvars:
        p = 0.0
        if g > 0:
            p = float(g) / n
            acc_entropy += entropy([p, 1.0 - p])

    return acc_entropy / len(grid_counters), grid_counters


def jensen_shannon_divergence(P, Q):
    if np.any(P < 0) or np.any(Q < 0):
        raise ValueError('Negative values.')
    if len(P) != len(Q):
        raise ValueError('Non equal size.')

    P_ = P / np.sum(P)      # Ensure probabilities.
    Q_ = Q / np.sum(Q)

    e1 = entropy(P_, base=2)
    e2 = entropy(Q_, base=2)
    e_sum = entropy((P_ + Q_) / 2.0, base=2)
    res = e_sum - ((e1 + e2) / 2.0)

    res2 = _jsdiv(P_, Q_)

    if not np.allclose(res, res2, atol=10e-5, rtol=0):
        warnings.warn('Numerical values of two JSD methods don\'t agree.')

    return res
    
def _jsdiv(P, Q):
    '''another way of computing JSD'''
    def _kldiv(A, B):
        a = A.copy()
        b = B.copy()
        idx = np.logical_and(a > 0, b > 0)
        a = a[idx]
        b = b[idx]
        return np.sum([v for v in a * np.log2(a / b)])

    P_ = P / np.sum(P)
    Q_ = Q / np.sum(Q)

    M = 0.5 * (P_ + Q_)

    return 0.5 * (_kldiv(P_, M) + _kldiv(Q_, M))




External Losses (Chamfer-EMD) cannot be loaded. Please install them first.


In [7]:
male_30 = np.load('/content/drive/MyDrive/Qmind Object Completion Team/TreeGAN/male_30_sample.npy')
male_30 = np.reshape(male_30, (30, 1, 3072, 3))

male_30.shape

(30, 1, 3072, 3)

In [8]:
print(jsd_between_point_cloud_sets(male_30[0], male_30[3]))

0.4661481256702622


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Point-clouds are not in unit cube.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Point-clouds are not in unit sphere.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Point-clouds are not in unit cube.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Point-clouds are not in unit sphere.


# COV metric

In [21]:
import math

def chamfer_distance(pointcloudA, pointcloudB):

  chamfer_distance = 0

  point_set = {}

  for point in pointcloudA:
    # Go through all the points in the other cloud, and add the smallest distance boy.
    smallest_distance = math.inf
    
    arr 

    for other_point in pointcloudB:
      dist = np.linalg.norm(point-other_point)
      if dist < smallest_distance:
        smallest_distance = dist
    


    chamfer_distance += smallest_distance

  for point in pointcloudB:
    # Go through all the points in the other cloud, and add the smallest distance boy.
    smallest_distance = math.inf
    for other_point in pointcloudA:
      dist = np.linalg.norm(point-other_point)
      if dist < smallest_distance:
        smallest_distance = dist
    chamfer_distance += smallest_distance

  return chamfer_distance


def coverage(sample_pcs, ref_pcs, distance_func):

  '''
  
  Recall that the coverage is to say 

  For each point cloud in A we first find its closest neighbor in B. Coverage is measured as the fraction of
  the point clouds in B that were matched to point clouds in
  A. Closeness can be computed using either the CD or EMD
  point-set distance of Section 2, thus yielding two different
  metrics, COV-CD and COV-EMD. A high coverage score
  indicates that most of B is roughly represented within A.
  
  '''

  # First dimension of each point cloud is going to be the len of the set of pointclouds (or technically the batch size)

  unique_indices = {} # changed to a hash table for speed improvement
  
  for i in range(len(sample_pcs)):
    A = sample_pcs[i]
    best_distance = math.inf
    best_index = 0
    for j in range(len(ref_pcs)):
      B = ref_pcs[j]
      distance = distance_func(A, B)
      if distance < best_distance:
        best_distance = distance
        best_index = j 
    
    if best_index not in unique_indices:
      unique_indices[best_index] = 1

  return len(unique_indices) / ref_pcs.shape[0]


## COV Unit Test

In [41]:
import time
start = time.time()

#A = male_30[0:2].reshape(2, 3072, 3)
A = male_30[0]
#B = male_30[3:5].reshape(2, 3072, 3)
B = male_30[3]

print("shape1", A.shape)
print("shape2", B.shape)

print(coverage(A, B, chamfer_distance))
end = time.time()

print("The time: ", end-start)


shape1 (1, 3072, 3)
shape2 (1, 3072, 3)
1.0
The time:  114.41469240188599


# MMD metric


## MMD Unit Test